In [12]:
## Started on May 7, 2021 at 1 PM CT
## Goal: don't use pyspeckit right now

# Import statements
%matplotlib inline
from spectral_cube import SpectralCube
from spectral_cube.dask_spectral_cube import DaskSpectralCube
from astroquery.splatalogue import Splatalogue
from astropy import units as u
from astropy import constants
from lte_modeling_tools import nupper_of_kkms
from pylab import imshow
import numpy as np
from scipy.optimize import curve_fit
import time
import matplotlib.pyplot as plt
import pylab as pl
import pyspeckit
from astropy.visualization import simple_norm
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.visualization import quantity_support
quantity_support()

# Check some things
import sys
sys.executable
import dask
dask.__version__ # My dask version was old but now it is updated

'2021.04.1'

In [7]:
# Import cube
results = '/blue/adamginsburg/abulatek/brick/symlinks/imaging_results/'
freq_spw = '146_spw51'
fn = results+'source_ab_'+freq_spw+'_clean_2sigma_n50000_masked_3sigma_pbmask0p18.image'
cube = SpectralCube.read(fn, format='casa_image', use_dask=True) # Import cube
freq_lo, freq_hi = 147.01*u.GHz, 147.17*u.GHz # Rough frequencies to search between
rough_subcube = cube.spectral_slab(freq_lo, freq_hi) # Generate rough subcube from cube
rough_subcube = rough_subcube.to(u.K) # Convert from Jy/beam to brightness temperature
print(rough_subcube) # Print metadata about rough subcube

/blue/adamginsburg/abulatek/anaconda/lib/python3.7/site-packages/casa_formats_io-0.1-py3.7-linux-x86_64.egg/casa_formats_io/casa_dask.py:232: RuntimeWarning: divide by zero encountered in long_scalars
  factors = [f for f in range(stacks[dim] + 1) if stacks[dim] % f == 0]


DaskVaryingResolutionSpectralCube with shape=(329, 512, 512) and unit=K and chunk size (255, 128, 128):
 n_x:    512  type_x: RA---SIN  unit_x: deg    range:   266.528130 deg:  266.560501 deg
 n_y:    512  type_y: DEC--SIN  unit_y: deg    range:   -28.719152 deg:  -28.690763 deg
 n_s:    329  type_s: FREQ      unit_s: Hz     range: 147009887391.829 Hz:147170048296.150 Hz


In [8]:
# Calculate the MAD-estimated RMS value for the rough subcube (can't do on original cube)
start = time.time()
sigma = np.nanmedian(rough_subcube.mad_std(axis = 0))
# sigma_subcube = sigma/np.sqrt(rough_subcube.shape[0]*rough_subcube.shape[1]*rough_subcube.shape[2])
sigma_subcube = sigma # Try this way; root(N) slopes are... very bad (and I believe 0.05 K over 1e-6 K)
print(sigma_subcube)
print("Sigma calculation:", time.time() - start, "seconds")

0.06657724331725512 K
Sigma calculation: 46.701669692993164 seconds


In [10]:
# List lines in this wavelength region, with tolerance for velocity shifts
tbl = Splatalogue.query_lines(freq_lo-0.1*u.GHz, freq_hi+0.1*u.GHz, 
                              chemical_name='CH3CN',
                              energy_max=500, # more lines w/ max energy > 140
                              energy_type='eu_k',
                              line_lists=['JPL'],
                              show_upper_degeneracy=True, 
                              show_qn_code=True)

tbl_trimmed = tbl[tbl['Quantum Number Code']==202]
# tbl_trimmed = tbl_trimmed[::-1]

# Retrieve values for later use
line_freqs = tbl_trimmed['Freq-GHz(rest frame,redshifted)'].data
line_names = tbl_trimmed['Resolved QNs'].data
einstein_A_coefficients = tbl_trimmed['Log<sub>10</sub> (A<sub>ij</sub>)'].data
E_upper = tbl_trimmed['E_U (K)'].data
g = tbl_trimmed['Upper State Degeneracy'].data

tbl_trimmed.show_in_notebook()

idx,Species,Chemical Name,"Freq-GHz(rest frame,redshifted)","Freq Err(rest frame,redshifted)","Meas Freq-GHz(rest frame,redshifted)","Meas Freq Err(rest frame,redshifted)",Resolved QNs,CDMS/JPL Intensity,S<sub>ij</sub>&#956;<sup>2</sup> (D<sup>2</sup>),S<sub>ij</sub>,Log<sub>10</sub> (A<sub>ij</sub>),Lovas/AST Intensity,E_L (cm^-1),E_L (K),E_U (cm^-1),E_U (K),Upper State Degeneracy,Quantum Number Code,Linelist
0,CH3CNv=0,Methyl Cyanide,147.0358351,1e-07,--,--,8(7)-7(7),-3.7949,39.74307,2.58376,-4.36401,--,260.3123,374.53377,265.21689,381.59042,34.0,202,JPL
1,CH3CNv=0,Methyl Cyanide,147.0726021,1e-07,--,--,8(6)-7(6),-3.0883,148.38336,9.64665,-4.09259,--,195.8526,281.79004,200.75841,288.84846,68.0,202,JPL
2,CH3CNv=0,Methyl Cyanide,147.103738,1e-07,--,--,8(5)-7(5),-3.1316,103.33459,6.71796,-3.94843,--,141.2849,203.27878,146.19175,210.33869,34.0,202,JPL
3,CH3CNv=0,Methyl Cyanide,147.1292302,1e-07,--,--,8(4)-7(4),-2.9482,127.19471,8.26914,-3.85798,--,96.6217,139.01798,101.5294,146.07911,34.0,202,JPL
4,CH3CNv=0,Methyl Cyanide,147.1490683,1e-07,--,--,8(3)-7(3),-2.5156,291.45821,18.94819,-3.79873,--,61.8731,89.02217,66.78146,96.08426,68.0,202,JPL
5,CH3CNv=0,Methyl Cyanide,147.1632441,2e-07,--,--,8(2)-7(2),-2.727,158.99516,10.33654,-3.76076,--,37.047,53.30272,41.95584,60.36548,34.0,202,JPL
6,CH3CNv=0,Methyl Cyanide,147.1717519,2e-07,--,--,8(1)-7(1),-2.6748,166.91824,10.85164,-3.73957,--,22.1491,31.86782,27.05822,38.931,34.0,202,JPL
7,CH3CNv=0,Methyl Cyanide,147.1745883,2e-07,--,--,8(0)-7(0),-2.6575,169.60673,11.02642,-3.7326,--,17.1828,24.72238,22.09202,31.78569,34.0,202,JPL


In [16]:
pcube = pyspeckit.Cube(cube = rough_subcube)

ValueError: For DaskVaryingResolutionSpectralCube's, use hdulist instead of hdu.